In [64]:
import numpy as np
import pandas as pd
import sklearn as sk
import xgboost as xgb

from sklearn.grid_search import GridSearchCV

def find_index(value, li):
    l = len(li)
    for i in range(l):
        if(li[i] == value):
            return i
    return -1


def find_best_value_for_parameter(X, y, other_parameter_values,
                                  parameter_name,
                                 first_level_values,
                                 second_level_values):
    grid = {parameter_name: first_level_values}
    clf = xgb.XGBClassifier()
    clf.set_params(**other_parameter_values)
    grid_search = GridSearchCV(estimator = clf, param_grid = grid, scoring='roc_auc', cv=5)
    grid_search.fit(X, y)
    ind = find_index(grid_search.best_params_[parameter_name], first_level_values)
    if(ind == -1):
        return grid_search.best_params_[parameter_name]
    else:
        grid = {parameter_name: second_level_values[ind]}
        grid_search = GridSearchCV(estimator = clf, param_grid = grid, scoring='roc_auc', cv=5)
        grid_search.fit(X, y)
        return grid_search.best_params_[parameter_name]
    
    
if __name__ == "__main__": 
    # load labeled data
    train_df = pd.read_csv('train.csv')
    train_data = pd.DataFrame.as_matrix(train_df)
    y = train_data[:,0]; X = train_data[:,1:9];
    from sklearn.preprocessing import OneHotEncoder
    enc = OneHotEncoder(categorical_features=[1])
    X = enc.fit_transform(X)

In [39]:
    # cross validation
    #grid = {'min_child_weight':range(1,6,2), 
    #        'colsample_bytree':[i/10.0 for i in range(6,8)],
    #        'max_delta_step':[i for i in range(1,3)]}
    #grid = {'max_delta_step':[i for i in range(1,3)],
    #       'gamma':[0,0.001,0.01,0.1,1,10]}
#     grid = {'n_estimators': [50,100,200,400,800]}
#     grid_search = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate =0.2, n_estimators=90, max_depth=9, objective= 'binary:logistic', seed=27), param_grid = grid, scoring='roc_auc', cv=5)
#     grid_search.fit(X, y)
#     print grid_search.best_params_
    other_parameter_values = {'learning_rate': 0.2,
                              'n_estimators':90,
                              'max_depth':9,
                              'objective':'binary:logistic',
                              'seed':27}
    parameter_name = 'n_estimators'
    other_parameter_values[parameter_name] = find_best_value_for_parameter(X, y, other_parameter_values,
                                                                          parameter_name,
                                                                          [50,100,200,400,800],
                                                                          {0:[50,60,70,80,90],
                                                                           1:[100,120,140,160,180],
                                                                           2:[200,240,280,320,360],
                                                                           3:[400,480,560,640,720],
                                                                           4:[800,880,960]})

In [ ]:
    print other_parameter_values[parameter_name]

In [ ]:
    parameter_name = 'max_depth'
    other_parameter_values[parameter_name] = find_best_value_for_parameter(X, y, other_parameter_values,
                                                                          parameter_name,
                                                                          [4, 8, 16, 32],
                                                                          {0:[4,5,6,7],
                                                                           1:[8,9,10,11,12,13,14,15],
                                                                           2:[16,20,24,28],
                                                                           3:[32, 40, 48, 56]})
    

In [ ]:
     print other_parameter_values[parameter_name]
    

In [ ]:
    parameter_name = 'colsample_bytree'
    other_parameter_values[parameter_name] = find_best_value_for_parameter(X, y, other_parameter_values,
                                                                          parameter_name,
                                                                          [0.3, 0.4, 0.5, 0.6, 0.7],
                                                                          {0:[0.3,0.32,0.34,0.36,0.38],
                                                                           1:[0.4,0.42,0.44,0.46,0.48],
                                                                           2:[0.5,0.52,0.54,0.56,0.58],
                                                                           3:[0.6,0.62,0.64,0.66,0.68],
                                                                           4:[0.7,0.72,0.74,0.76,0.78]}) 
    

In [ ]:
     print other_parameter_values[parameter_name]
    

In [ ]:
    parameter_name = 'n_estimators'
    other_parameter_values[parameter_name] = find_best_value_for_parameter(X, y, other_parameter_values,
                                                                          parameter_name,
                                                                          [50,100,200,400,800,1600,3200],
                                                                          {0:[50],
                                                                           1:[100],
                                                                           2:[200],
                                                                           3:[400],
                                                                           4:[800],
                                                                           5:[1600],
                                                                           6:[3200]})
    print other_parameter_values[parameter_name]
    

In [ ]:
    parameter_name = 'max_depth'
    other_parameter_values[parameter_name] = find_best_value_for_parameter(X, y, other_parameter_values,
                                                                          parameter_name,
                                                                          [4, 8, 16, 32],
                                                                          {0:[4,5,6,7],
                                                                           1:[8,9,10,11,12,13,14,15],
                                                                           2:[16,20,24,28],
                                                                           3:[32, 40, 48, 56]})
    print other_parameter_values[parameter_name] 
    

In [75]:
    parameter_name = 'colsample_bytree'
    other_parameter_values[parameter_name] = find_best_value_for_parameter(X, y, other_parameter_values,
                                                                          parameter_name,
                                                                          [0.1, 0.2, 0.3],
                                                                          {0:[0.1],
                                                                           1:[0.2],
                                                                           2:[0.3]}) 
    

In [76]:
    print other_parameter_values[parameter_name]

0.2


In [77]:
    # train with best parameters
#     xgb_clsf = xgb.XGBClassifier(learning_rate =0.2,
#                                  n_estimators=100, 
#                                  max_depth=9, objective= 'binary:logistic',
#                                 #min_child_weight = grid_search.best_params_['min_child_weight'],
#                                 #colsample_bytree=grid_search.best_params_['colsample_bytree'],
#                                  min_child_weight = 1,
#                                  colsample_bytree = 0.4)
#                                  #gamma = 0.0,
#                                  #max_delta_step=2)

    
    xgb_clsf = xgb.XGBClassifier()
    xgb_clsf.set_params(**other_parameter_values)
    xgb_clsf.fit(X, y)
    
    print other_parameter_values

{'colsample_bytree': 0.2, 'learning_rate': 0.2, 'n_estimators': 400, 'seed': 27, 'objective': 'binary:logistic', 'max_depth': 32}


In [78]:
    # load unlabeled data
    test_df = pd.read_csv('test.csv')
    test_data = pd.DataFrame.as_matrix(test_df)
    id_test = test_data[:,0]; X_test = test_data[:,1:9];
    X_test = enc.fit_transform(X_test)
    # predictions on unlabeled data
    y_test_pred = xgb_clsf.predict_proba(X_test)
    ans = pd.DataFrame({'Id': id_test, 'Action' : y_test_pred[:,1]})
    ans.to_csv('hw3p2.csv', index=False, columns=['Id', 'Action'])
    

In [69]:
    print X_test

  (0, 4080)	1.0
  (1, 642)	1.0
  (2, 295)	1.0
  (3, 2227)	1.0
  (4, 3176)	1.0
  (5, 186)	1.0
  (6, 2327)	1.0
  (7, 401)	1.0
  (8, 1787)	1.0
  (9, 660)	1.0
  (10, 2547)	1.0
  (11, 1036)	1.0
  (12, 3439)	1.0
  (13, 4461)	1.0
  (14, 173)	1.0
  (15, 2209)	1.0
  (16, 3515)	1.0
  (17, 228)	1.0
  (18, 540)	1.0
  (19, 2927)	1.0
  (20, 65)	1.0
  (21, 1175)	1.0
  (22, 347)	1.0
  (23, 922)	1.0
  (24, 3104)	1.0
  :	:
  (58917, 4692)	124948.0
  (58917, 4693)	118054.0
  (58917, 4694)	118054.0
  (58917, 4695)	117887.0
  (58918, 4689)	39262.0
  (58918, 4690)	117961.0
  (58918, 4691)	118343.0
  (58918, 4692)	6104.0
  (58918, 4693)	118278.0
  (58918, 4694)	118260.0
  (58918, 4695)	290919.0
  (58919, 4689)	1023.0
  (58919, 4690)	117961.0
  (58919, 4691)	118327.0
  (58919, 4692)	120318.0
  (58919, 4693)	118321.0
  (58919, 4694)	117906.0
  (58919, 4695)	290919.0
  (58920, 4689)	44977.0
  (58920, 4690)	117961.0
  (58920, 4691)	118343.0
  (58920, 4692)	118660.0
  (58920, 4693)	119433.0
  (58920, 4694)	133686

In [79]:
    print X_test.shape

(58921, 4696)
